In [65]:
import pandas as pd
import numpy as np
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


In [66]:
duplicates = diabetes[diabetes.duplicated()]
print('Число дубликтов: {}'.format(duplicates.shape[0]))
diabetes = diabetes.drop_duplicates()
print('Результирующее число записей: {}'.format(diabetes.shape[0]))

Число дубликтов: 10
Результирующее число записей: 768


Далее найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.95: удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны. В ответ запишите имена признаков, которые вы нашли (без кавычек).

In [67]:
list_stat=[]
for i in diabetes.columns:
    dupl=diabetes[i].value_counts(normalize=True).max()
    nunique_ratio = diabetes[i].nunique() / diabetes[i].count()
    if dupl > 0.95:
        list_stat.append(i)
        print(f'{i}: {round(dupl*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        list_stat.append(i)
        print(f'{i}: {round(nunique_ratio*100, 2)}% уникальных значений')

Gender: 100.0% одинаковых значений


In [68]:
diabetes=diabetes.replace(0, np.nan)

Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.

Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.

In [69]:
round(diabetes[diabetes['Insulin'].isna()].shape[0]/diabetes['Insulin'].shape[0],2)

0.49

Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?

In [70]:
list_stat=[]
for i in diabetes.columns:
    perc_nan=diabetes[diabetes[i].isna()].shape[0]/diabetes[i].shape[0]
    if perc_nan > 0.3:
        list_stat.append(i)
        print(f'{i}: {round(perc_nan*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
print(list_stat)

Insulin: 48.7% одинаковых значений
Outcome: 65.1% одинаковых значений
['Insulin', 'Outcome']


In [71]:
diabetes=diabetes.drop(axis=1, labels=list_stat)


In [72]:
diabetes.shape[1]
m=diabetes.shape[1]

Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?

In [73]:
diabetes=diabetes.dropna(axis=0, thresh=m-2)

In [74]:
diabetes.shape[0]

752

В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.

In [76]:
for i in diabetes.columns:
    try:
        diabetes[i].fillna(diabetes[i].median())
    except TypeError:
        continue
    else:
        diabetes[i]=diabetes[i].fillna(diabetes[i].median())

diabetes

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Gender
0,6.0,98.0,58.0,33.0,34.0,0.430,43,Female
1,2.0,112.0,75.0,32.0,35.7,0.148,21,Female
2,2.0,108.0,64.0,29.0,30.8,0.158,21,Female
3,8.0,107.0,80.0,29.0,24.6,0.856,34,Female
4,7.0,136.0,90.0,29.0,29.9,0.210,50,Female
...,...,...,...,...,...,...,...,...
762,2.0,122.0,76.0,27.0,35.9,0.483,26,Female
763,5.0,139.0,64.0,35.0,28.6,0.411,26,Female
764,1.0,96.0,122.0,29.0,22.4,0.207,27,Female
765,10.0,101.0,86.0,37.0,45.6,1.136,38,Female


In [77]:
diabetes['SkinThickness'].mean()

29.11037234042553

Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?

In [81]:
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, _ = outliers_iqr_mod(diabetes, 'SkinThickness')
print(outliers.shape[0])

81


Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?

In [83]:
def outliers_z_score(data, feature, left=3, right=3, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    mean = x.mean()
    sqr = x.std()
    lower_bound = mean - sqr*left
    upper_bound = mean + sqr*right
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, _ = outliers_z_score(diabetes, 'SkinThickness')
print(outliers.shape[0])

4


In [84]:
outliers, _ = outliers_iqr_mod(diabetes, 'DiabetesPedigreeFunction')
print(outliers.shape[0])

11


In [85]:
outliers, _ = outliers_iqr_mod(diabetes, 'DiabetesPedigreeFunction', log_scale=True)
print(outliers.shape[0])

0
